In [ ]:
%cd ..

In [ ]:
from apperception.database import database

In [ ]:
### Add Road Direction Attributes to Tables ###
query = """ALTER TABLE Item_General_Trajectory
           ADD roadDirection tfloat;

           ALTER TABLE Cameras
           ADD roadDirection numeric;
        """
database._execute_query(query=query)

In [ ]:
### Update Road Direction For Camera ###
query = """UPDATE Cameras AS c
           SET roadDirection = roadDirection(egoTranslation, egoHeading)
           WHERE roadDirection IS NULL;
        """
database._execute_query(query=query)

In [ ]:
### Create Function For roadDirection With Traj ###
query = """ 
        DROP FUNCTION IF EXISTS _roadDirectionAttr(trajectory tgeompoint, headings tfloat);
        CREATE OR REPLACE FUNCTION _roadDirectionAttr(trajectory tgeompoint, headings tfloat) RETURNS tfloat AS
        $BODY$
        BEGIN
        RETURN roadDirection(valueAtTimestamp(trajectory, _time), default_dir);
        END
        $BODY$
        LANGUAGE 'plpgsql' ;
        """